In [51]:
# Cell 1: Настройка путей, импорт и загрузка функций инференса
import os
import sys
import glob
import importlib

import torch
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms

# Корень проекта — две папки выше текущего рабочего каталога
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Путь до папки inference, чтобы Python видел deep_text_recognition и ocr_inference
inference_dir = os.path.join(project_root, 'custom_inference')
if inference_dir not in sys.path:
    sys.path.insert(0, inference_dir)

# Импортируем наш модуль инференса
import ocr_inference  # он загружен из inference/ocr_inference.py

# Забираем функции из него
load_model_and_converter = ocr_inference.load_model_and_converter
predict_text             = ocr_inference.predict_text

# Пути к моделям и данным
MODELS_DIR = os.path.join(inference_dir, 'models', 'ocr')
IMAGES_DIR = os.path.join(project_root, 'dataset', 'crops')
CSV_PATH   = os.path.join(IMAGES_DIR, 'ocr_data.csv')

# Пути до чекпойнтов
pth1 = os.path.join(MODELS_DIR, 'TPS-ResNet-BiLSTM-Attn.pth')
pth2 = os.path.join(MODELS_DIR, 'best_accuracy.pth')

print("Cell 1: Импорт выполнен, ready to proceed.")


Cell 1: Импорт выполнен, ready to proceed.


In [52]:
# Cell 2: Определяем opt1 и opt2 точно так, как при обучении
class Opt1:
    pass

opt1 = Opt1()
opt1.imgH = 64
opt1.imgW = 50
opt1.input_channel     = 1
opt1.output_channel    = 512
opt1.hidden_size       = 256
opt1.num_fiducial      = 20
# Словарь длины 36 → num_class = 38
opt1.character = '0123456789abcdefghijklmnopqrstuvwxyz'  # len=36
opt1.num_class   = len(opt1.character) + 2              # = 38
opt1.Transformation    = 'TPS'
opt1.FeatureExtraction = 'ResNet'
opt1.SequenceModeling  = 'BiLSTM'
opt1.Prediction        = 'Attn'
opt1.batch_max_length  = 25

class Opt2:
    pass

opt2 = Opt2()
opt2.imgH = 100
opt2.imgW = 100
opt2.input_channel     = 1
opt2.output_channel    = 512
opt2.hidden_size       = 256
opt2.num_fiducial      = 20
# Словарь длины 10 → num_class = 12
opt2.character = '0123456789'
opt2.num_class   = len(opt2.character) + 2  # = 12
opt2.Transformation    = 'TPS'
opt2.FeatureExtraction = 'ResNet'
opt2.SequenceModeling  = 'BiLSTM'
opt2.Prediction        = 'Attn'
opt2.batch_max_length  = 25

print("Cell 2: Определены конфигурации opt1 и opt2.")


Cell 2: Определены конфигурации opt1 и opt2.


In [53]:
# Cell 3: Загрузка CSV с аннотациями и формирование списка изображений
df = pd.read_csv(CSV_PATH)
print("Первые 5 строк CSV:")
display(df.head())

# Собираем DataFrame с полным путём и ground-truth
data = []
for _, row in df.iterrows():
    fname = row['filename']
    gt    = str(row['words'])
    img_path = os.path.join(IMAGES_DIR, fname)
    if os.path.isfile(img_path):
        data.append({'filename': fname, 'gt': gt, 'path': img_path})
data = pd.DataFrame(data)
print(f"Найдено {len(data)} изображений.")
display(data.head())


Первые 5 строк CSV:


,filename,words
0,crop_1006.png,1
1,crop_1007.png,1
2,crop_1008.png,1
3,crop_1009.png,1
4,crop_1010.png,1


Найдено 575 изображений.


,filename,gt,path
0,crop_1006.png,1,c:\Users\thjat\ml-system-design-aith\dataset\c...
1,crop_1007.png,1,c:\Users\thjat\ml-system-design-aith\dataset\c...
2,crop_1008.png,1,c:\Users\thjat\ml-system-design-aith\dataset\c...
3,crop_1009.png,1,c:\Users\thjat\ml-system-design-aith\dataset\c...
4,crop_1010.png,1,c:\Users\thjat\ml-system-design-aith\dataset\c...


In [54]:
# Cell 4: Функция для подсчёта расстояния Левенштейна
def levenshtein_distance(a: str, b: str) -> int:
    if a == b:
        return 0
    if len(a) == 0:
        return len(b)
    if len(b) == 0:
        return len(a)
    n, m = len(a), len(b)
    dp = np.zeros((n + 1, m + 1), dtype=int)
    for i in range(n + 1):
        dp[i, 0] = i
    for j in range(m + 1):
        dp[0, j] = j
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = 0 if a[i - 1] == b[j - 1] else 1
            dp[i, j] = min(
                dp[i - 1, j] + 1,
                dp[i, j - 1] + 1,
                dp[i - 1, j - 1] + cost
            )
    return int(dp[n, m])

print("Cell 4: Функция Левенштейна готова.")


Cell 4: Функция Левенштейна готова.


In [58]:
# Cell 5: Переписанная загрузка модели + создание конвертера (CTC/Attn) без "module." в state_dict
from custom_inference.deep_text_recognition.model import Model
from custom_inference.deep_text_recognition.utils import AttnLabelConverter, CTCLabelConverter


def load_model_and_converter_strip(opt, weights_path: str, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device(device)
    model = Model(opt).to(device)

    raw_sd = torch.load(weights_path, map_location=device)
    stripped_sd = {}
    for k, v in raw_sd.items():
        new_key = k.replace("module.", "") if k.startswith("module.") else k
        stripped_sd[new_key] = v

    model.load_state_dict(stripped_sd)
    model.eval()

    if opt.Prediction == 'CTC':
        converter = CTCLabelConverter(opt.character)
    elif opt.Prediction == 'Attn':
        converter = AttnLabelConverter(opt.character)
    else:
        raise ValueError(f"Unknown Prediction: {opt.Prediction}")

    return model, converter

# Тест загрузки для обеих моделей
try:
    model1, converter1 = load_model_and_converter_strip(opt1, pth1)
    device1 = next(model1.parameters()).device
    print(f"Первая модель загружена на {device1}")
except Exception as e:
    print("Ошибка при загрузке первой модели:", e)

try:
    model2, converter2 = load_model_and_converter_strip(opt2, pth2)
    device2 = next(model2.parameters()).device
    print(f"Вторая модель загружена на {device2}")
except Exception as e:
    print("Ошибка при загрузке второй модели:", e)


Первая модель загружена на cpu
Вторая модель загружена на cpu


In [62]:
# Cell 6: Исправленная функция predict_text для корректного вызова forward (добавляем text для Attn)
def predict_text_fixed(model: torch.nn.Module, converter, frame: np.ndarray, opt) -> str:
    device = next(model.parameters()).device

    # 1) BGR -> RGB -> PIL
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(frame_rgb)

    # 2) Каналы
    if opt.input_channel == 1:
        pil_img = pil_img.convert('L')
    else:
        pil_img = pil_img.convert('RGB')

    # 3) Препроцессинг: Resize, ToTensor, Normalize
    transform_list = [
        transforms.Resize((opt.imgH, opt.imgW)),
        transforms.ToTensor()
    ]
    if opt.input_channel == 1:
        transform_list.append(transforms.Normalize(mean=[0.5], std=[0.5]))
    else:
        transform_list.append(transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                   std=[0.5, 0.5, 0.5]))
    preprocess = transforms.Compose(transform_list)

    input_tensor = preprocess(pil_img)                  # [C, H, W]
    input_batch  = input_tensor.unsqueeze(0).to(device)  # [1, C, H, W]

    # 4) Forward: для Attn нужен текстовый ввод; для CTC — нет
    with torch.no_grad():
        if opt.Prediction == 'Attn':
            # создаём текстовый тензор: [batch=1, max_length]
            text_input = torch.zeros((1, opt.batch_max_length), dtype=torch.long).to(device)
            text_input[:, 0] = 0  # <SOS> в первой позиции
            output = model(input_batch, text_input, False)
        else:  # CTC
            output = model(input_batch)

    # 5) Декодинг
    if opt.Prediction == 'CTC':
        logits = output.log_softmax(2)          # [1, seq_len, num_classes]
        _, preds_index = logits.max(2)          # [1, seq_len]
        preds_index = preds_index.view(1, -1)

        seq_length = preds_index.size(1)
        length_for_pred = torch.IntTensor([seq_length]).to(device)

        preds_str_list = converter.decode(preds_index, length_for_pred)
        preds_str = preds_str_list[0]

    else:  # Attn
        _, preds_index = output.max(2)         # [1, max_length]
        preds_index = preds_index.view(1, -1)

        length_for_pred = torch.IntTensor([opt.batch_max_length]).to(device)
        preds_str_list = converter.decode(preds_index, length_for_pred)
        preds_str = preds_str_list[0]

    preds_str = preds_str.replace('[s]', '')
    return preds_str

# Проверим predict_text_fixed на одной картинке
sample_path = data.loc[0, 'path']
img_bgr = cv2.imread(sample_path)
print(f"Тестовая картинка: {sample_path}, shape = {img_bgr.shape}")

pred1 = predict_text_fixed(model1, converter1, img_bgr, opt1)
print("Первое предсказание (модель 1):", repr(pred1))

pred2 = predict_text_fixed(model2, converter2, img_bgr, opt2)
print("Первое предсказание (модель 2):", repr(pred2))


Тестовая картинка: c:\Users\thjat\ml-system-design-aith\dataset\crops\crop_1006.png, shape = (172, 104, 3)
Первое предсказание (модель 1): 'liedlylyedly'
Первое предсказание (модель 2): '1'


In [63]:
# Cell 7: Полный inference по всем изображениям для обеих моделей с predict_text_fixed
results = {}

for model_name, (opt, model, converter) in [
    ('TPS-ResNet-BiLSTM-Attn', (opt1, model1, converter1)),
    ('best_accuracy',           (opt2, model2, converter2))
]:
    print(f"\n=== Инференс моделью: {model_name} ===")
    preds_for_model = []
    for _, row in data.iterrows():
        img_path = row['path']
        img_bgr  = cv2.imread(img_path)
        if img_bgr is None:
            preds_for_model.append({'filename': row['filename'], 'pred': '', 'lev_dist': None})
            continue
        pred_text = predict_text_fixed(model, converter, img_bgr, opt)
        lev = levenshtein_distance(pred_text, row['gt'])
        preds_for_model.append({
            'filename': row['filename'],
            'pred': pred_text,
            'lev_dist': lev
        })
    results[model_name] = pd.DataFrame(preds_for_model)
    print(f"Первые 5 предсказаний для {model_name}:")
    display(results[model_name].head())



=== Инференс моделью: TPS-ResNet-BiLSTM-Attn ===
Первые 5 предсказаний для TPS-ResNet-BiLSTM-Attn:


,filename,pred,lev_dist
0,crop_1006.png,liedlylyedly,12
1,crop_1007.png,letingy,7
2,crop_1008.png,lingedlylyedly,14
3,crop_1009.png,linedlyly,9
4,crop_1010.png,lingedesly,10



=== Инференс моделью: best_accuracy ===
Первые 5 предсказаний для best_accuracy:


,filename,pred,lev_dist
0,crop_1006.png,1,0
1,crop_1007.png,1,0
2,crop_1008.png,1,0
3,crop_1009.png,1,0
4,crop_1010.png,1,0


In [64]:
# Cell 8: Объединяем результаты в одну таблицу и сравниваем
compare_df = data[['filename', 'gt']].copy()
for model_name, df_pred in results.items():
    compare_df = compare_df.merge(
        df_pred.rename(columns={'pred': f'pred_{model_name}', 'lev_dist': f'lev_{model_name}'}),
        on='filename', how='left'
    )

print("Сводная таблица (первые 10 строк):")
display(compare_df.head(10))


Сводная таблица (первые 10 строк):


,filename,gt,pred_TPS-ResNet-BiLSTM-Attn,lev_TPS-ResNet-BiLSTM-Attn,pred_best_accuracy,lev_best_accuracy
0,crop_1006.png,1,liedlylyedly,12,1,0
1,crop_1007.png,1,letingy,7,1,0
2,crop_1008.png,1,lingedlylyedly,14,1,0
3,crop_1009.png,1,linedlyly,9,1,0
4,crop_1010.png,1,lingedesly,10,1,0
5,crop_1011.png,1,letingesylye,12,1,0
6,crop_1012.png,2,letingyesylys,13,2,0
7,crop_1013.png,2,letingylyses,12,2,0
8,crop_1014.png,2,letingyeeslyly,14,2,0
9,crop_1015.png,2,listedlylyse,12,2,0


In [65]:
# Cell 9: Подсчёт итоговых метрик: точность слова и средняя Левенштейна
metrics = []
for model_name in results.keys():
    pred_col = f'pred_{model_name}'
    lev_col  = f'lev_{model_name}'
    total = len(compare_df)
    exact_match = (compare_df[lev_col] == 0).sum()
    word_acc = exact_match / total if total > 0 else 0.0
    valid_lev = compare_df[lev_col].dropna()
    mean_lev = valid_lev.mean() if len(valid_lev) > 0 else float('nan')
    metrics.append({
        'model': model_name,
        'total_images': total,
        'exact_matches': int(exact_match),
        'word_accuracy': float(word_acc),
        'mean_levenshtein': float(mean_lev)
    })

metrics_df = pd.DataFrame(metrics)
print("Итоговые метрики по моделям:")
display(metrics_df)


Итоговые метрики по моделям:


,model,total_images,exact_matches,word_accuracy,mean_levenshtein
0,TPS-ResNet-BiLSTM-Attn,575,0,0.000000,10.326957
1,best_accuracy,575,558,0.970435,0.033043


In [66]:
# Cell 10 (опционально): Сохранение результатов в CSV
out_dir = os.path.join(project_root, 'training', 'comparision')
os.makedirs(out_dir, exist_ok=True)
compare_df.to_csv(os.path.join(out_dir, 'ocr_comparison_results.csv'), index=False)
metrics_df.to_csv(os.path.join(out_dir, 'ocr_comparison_metrics.csv'), index=False)
print(f"Результаты сохранены в папке {out_dir}")


Результаты сохранены в папке c:\Users\thjat\ml-system-design-aith\training\comparision


In [67]:
# Cell 11: Итоговые метрики (word accuracy, средняя Левенштейна) для обеих моделей
metrics = []
for model_name, df_pred in results.items():
    if df_pred is None:
        continue
    pred_col = f'pred_{model_name}'
    lev_col  = f'lev_{model_name}'

    total = len(compare_df)
    exact_match = (compare_df[lev_col] == 0).sum()
    acc = exact_match / total if total > 0 else 0.0

    valid_lev = compare_df[lev_col].dropna()
    mean_lev = valid_lev.mean() if len(valid_lev) > 0 else np.nan

    metrics.append({
        'model': model_name,
        'total_images': total,
        'exact_matches': int(exact_match),
        'word_accuracy': float(acc),
        'mean_levenshtein': float(mean_lev)
    })

metrics_df = pd.DataFrame(metrics)
print("Итоговые метрики:")
display(metrics_df)

Итоговые метрики:


,model,total_images,exact_matches,word_accuracy,mean_levenshtein
0,TPS-ResNet-BiLSTM-Attn,575,0,0.000000,10.326957
1,best_accuracy,575,558,0.970435,0.033043


In [69]:
# Cell 12 (опционально): Сохраняем результаты
os.makedirs(os.path.join(project_root, 'training', 'comparsion'), exist_ok=True)
compare_df.to_csv(os.path.join(project_root, 'training', 'comparsion', 'ocr_comparsion_results.csv'), index=False)
metrics_df.to_csv(os.path.join(project_root, 'training', 'comparsion', 'ocr_comparsion_metrics.csv'), index=False)
print("Результаты и метрики сохранены в папку training/comparision/")

Результаты и метрики сохранены в папку training/comparision/
